In [1]:
# for presentation purposes
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import prepare

# visualize 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# working with dates
from datetime import datetime

# to evaluated performance using rmse
from sklearn.metrics import mean_squared_error
from math import sqrt 

# for tsa 
import statsmodels.api as sm

# holt's linear trend model. 
from statsmodels.tsa.api import Holt

In [12]:
# plotting defaults
plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=16)

#### Load Sales_Items_Stores data

In [2]:
# using store_items_sales df which is my merged csv
df = prepare.stores_convert_datetime()

In [3]:
df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14,item,sale_amount,sale_date,sale_id,store,store_address,store_city,store_id,store_state,store_zipcode
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,1,13.0,2013-01-01,1,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,1,11.0,2013-01-02,2,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
2,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,1,14.0,2013-01-03,3,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
3,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,1,13.0,2013-01-04,4,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
4,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,1,10.0,2013-01-05,5,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [5]:
merged = prepare.set_datetime_index(df, 'sale_date')

In [6]:
merged.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14,item,sale_amount,sale_id,store,store_address,store_city,store_id,store_state,store_zipcode
sale_date,,,,,,,,,,,,,,,
2013-01-01,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,1,13.0,1,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
2013-01-01,Ducal,17,Ducal Refried Red Beans,1.16,88313590791,88313590791,17,26.0,295813,3,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
2013-01-01,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004,7,32.0,125995,10,8503 NW Military Hwy,San Antonio,10,TX,78231
2013-01-01,Scotch,18,Scotch Removable Clear Mounting Squares - 35 Ct,4.39,21200725340,21200725340,18,45.0,314073,3,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
2013-01-01,Careone,19,Careone Family Comb Set - 8 Ct,0.74,41520035646,41520035646,19,34.0,332333,3,2118 Fredericksburg Rdj,San Antonio,3,TX,78201


In [7]:
merged = prepare.add_sales_total(merged)

In [8]:
merged.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14,item,sale_amount,sale_id,store,store_address,store_city,store_id,store_state,store_zipcode,sales_total
sale_date,,,,,,,,,,,,,,,,
2013-01-01,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,1,13.0,1,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,15.476190
2013-01-01,Ducal,17,Ducal Refried Red Beans,1.16,88313590791,88313590791,17,26.0,295813,3,2118 Fredericksburg Rdj,San Antonio,3,TX,78201,22.413793
2013-01-01,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004,7,32.0,125995,10,8503 NW Military Hwy,San Antonio,10,TX,78231,3.319502
2013-01-01,Scotch,18,Scotch Removable Clear Mounting Squares - 35 Ct,4.39,21200725340,21200725340,18,45.0,314073,3,2118 Fredericksburg Rdj,San Antonio,3,TX,78201,10.250569
2013-01-01,Careone,19,Careone Family Comb Set - 8 Ct,0.74,41520035646,41520035646,19,34.0,332333,3,2118 Fredericksburg Rdj,San Antonio,3,TX,78201,45.945946


In [14]:
merged = merged.index.astype('date')

dtype('O')

1) Split data (train/validate/test) and resample by any period, except daily, and aggregate using the sum.

In [33]:
# resample by month
df_M = train.resample('M')[['sales_total', 'sale_amount']].sum()

In [34]:
df_M.head()

,sales_total,sale_amount
sale_date,,
2013-01-31,137827.325902,454904.0
2013-02-28,139049.208221,459417.0
2013-03-31,186595.118651,617382.0
2013-04-30,206431.192464,682274.0
2013-05-31,231277.447129,763242.0


In [31]:
df_M = df_M.rename(columns={'sales_total': 'units_sold'})

In [36]:
train = df_M[:'2016'] # includes 2016
test = df_M['2017']

plt.plot(train.index, train.sale_amount)
plt.plot(test.index, test.sale_amount)

KeyError: '2017'